# Scrubbing and Cleaning Data - Lab

## Introduction

In the previous labs, you joined the data from our separate files into a single DataFrame.  In this lab, you'll scrub the data to get it ready for exploration and modeling!

## Objectives

You will be able to:

* Cast columns to the appropriate data types
* Identify and deal with null values appropriately
* Remove unnecessary columns
* Understand how to normalize data


## Getting Started

You'll find the resulting dataset from our work in the _Obtaining Data_ Lab stored within the file `walmart_data_not_cleaned.csv`.  

In the cells below:

* Import pandas and set the standard alias
* Import numpy and set the standard alias
* Import matplotlib.pyplot and set the standard alias
* Import seaborn and set the alias `sns` (this is the standard alias for seaborn)
* Use the ipython magic command to set all matplotlib visualizations to display inline in the notebook
* Load the dataset stored in the .csv file into a DataFrame using pandas
* Inspect the head of the DataFrame to ensure everything loaded correctly

In [1]:
# Import statements go here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [2]:
df = pd.read_csv("Lego_data_merged.csv")

## Starting our Data Cleaning

To start, you'll deal with the most obvious issue: data features with the wrong data encoding.

### Checking Data Types

In the cell below, use the appropriate method to check the data type of each column. 

In [3]:
df.dtypes

prod_id                int64
ages                  object
piece_count            int64
set_name              object
prod_desc             object
prod_long_desc        object
theme_name            object
country               object
list_price            object
num_reviews          float64
play_star_rating     float64
review_difficulty     object
star_rating          float64
val_star_rating      float64
dtype: object

Now, investigate some of the unique values inside of the `list_price` column.

In [4]:
df["list_price"] = df["list_price"].apply(lambda x: round(float(x[1:]),2))

In [5]:
df["list_price"] = df["list_price"].astype(float)

### Numerical Data Stored as Strings

A common issue to check for at this stage is numeric columns that have accidentally been encoded as strings. For example, you should notice that the `list_price` column above is currently formatted as a string and contains a proceeding '$'. Remove this and convert the remaining number to a `float` so that you can later model this value. After all, your primary task is to generate model to predict the price.

> Note: While the data spans a multitude of countries, assume for now that all prices have been standardized to USD.

In [6]:
#Your code here

### Detecting and Dealing With Null Values

Next, it's time to check for null values. How to deal with the null values will be determined by the columns containing them, and how many null values exist in each.  
 
In the cell below, get a count of how many null values exist in each column in the DataFrame. 

In [7]:
df.head()

,prod_id,ages,piece_count,set_name,prod_desc,prod_long_desc,theme_name,country,list_price,num_reviews,play_star_rating,review_difficulty,star_rating,val_star_rating
0,75823,6-12,277,Bird Island Egg Heist,Catapult into action and take back the eggs fr...,Use the staircase catapult to launch Red into ...,Angry Birds™,US,29.99,2.0,4.0,Average,4.5,4.0
1,75822,6-12,168,Piggy Plane Attack,Launch a flying attack and rescue the eggs fro...,Pilot Pig has taken off from Bird Island with ...,Angry Birds™,US,19.99,2.0,4.0,Easy,5.0,4.0
2,75821,6-12,74,Piggy Car Escape,Chase the piggy with lightning-fast Chuck and ...,Pitch speedy bird Chuck against the Piggy Car....,Angry Birds™,US,12.99,11.0,4.3,Easy,4.3,4.1
3,21030,12+,1032,United States Capitol Building,Explore the architecture of the United States ...,Discover the architectural secrets of the icon...,Architecture,US,99.99,23.0,3.6,Average,4.6,4.3
4,21035,12+,744,Solomon R. Guggenheim Museum®,Recreate the Solomon R. Guggenheim Museum® wit...,Discover the architectural secrets of Frank Ll...,Architecture,US,79.99,14.0,3.2,Challenging,4.6,4.1


In [8]:
df.isna().sum()

prod_id                 0
ages                    0
piece_count             0
set_name                0
prod_desc             358
prod_long_desc          0
theme_name              0
country                 0
list_price              0
num_reviews          1421
play_star_rating     1549
review_difficulty    1766
star_rating          1421
val_star_rating      1569
dtype: int64

Now, get some descriptive statistics for each of the columns. You want to see where the minimum and maximum values lie.  

In [9]:
df['ages'].describe()

count     10870
unique       31
top        6-12
freq       1476
Name: ages, dtype: object

In [10]:
df['num_reviews'].describe()

count    9449.000000
mean       17.813737
std        38.166693
min         1.000000
25%         2.000000
50%         6.000000
75%        14.000000
max       367.000000
Name: num_reviews, dtype: float64

Now that you have a bit more of a understanding of each of these features you can now make an informed decision about the best strategy for dealing with the various null values. 

Some common strategies for filling null values include:
* Using the mean of the feature
* Using the median of the feature
* Inserting a random value from a normal distribution with the mean and std of the feature
* Binning

Given that most of the features with null values concern user reviews of the lego set, it is reasonable to wonder whether there is strong correlation between these features in the first place. Before proceeding, take a minute to investigate this hypothesis.

In [11]:
#Investigate whether multicollinearity exists between the review features 
#(num_reviews, play_star_rating, star_rating, val_star_rating)
from statsmodels.stats.outliers_influence import variance_inflation_factor    
from itertools import combinations

In [12]:
options = ['num_reviews', 'play_star_rating', 'star_rating', 'val_star_rating']
combs = combinations(options,2)
#variance_inflation_factor()

In [13]:
df.head()

,prod_id,ages,piece_count,set_name,prod_desc,prod_long_desc,theme_name,country,list_price,num_reviews,play_star_rating,review_difficulty,star_rating,val_star_rating
0,75823,6-12,277,Bird Island Egg Heist,Catapult into action and take back the eggs fr...,Use the staircase catapult to launch Red into ...,Angry Birds™,US,29.99,2.0,4.0,Average,4.5,4.0
1,75822,6-12,168,Piggy Plane Attack,Launch a flying attack and rescue the eggs fro...,Pilot Pig has taken off from Bird Island with ...,Angry Birds™,US,19.99,2.0,4.0,Easy,5.0,4.0
2,75821,6-12,74,Piggy Car Escape,Chase the piggy with lightning-fast Chuck and ...,Pitch speedy bird Chuck against the Piggy Car....,Angry Birds™,US,12.99,11.0,4.3,Easy,4.3,4.1
3,21030,12+,1032,United States Capitol Building,Explore the architecture of the United States ...,Discover the architectural secrets of the icon...,Architecture,US,99.99,23.0,3.6,Average,4.6,4.3
4,21035,12+,744,Solomon R. Guggenheim Museum®,Recreate the Solomon R. Guggenheim Museum® wit...,Discover the architectural secrets of Frank Ll...,Architecture,US,79.99,14.0,3.2,Challenging,4.6,4.1


In [14]:
def vif(col1, col2, df):
    from statsmodels.formula.api import ols
    r_squared_i = ols("%s~%s" % (col1, col2), data=df).fit().rsquared
    vif = 1. / (1. - r_squared_i)
    return vif

In [15]:

for comb1, comb2 in combinations(options,2):
    print(comb1, comb2, vif(comb1, comb2, df))

num_reviews play_star_rating 1.0037207024262647
num_reviews star_rating 1.0000206192943422
num_reviews val_star_rating 1.0007114680797857
play_star_rating star_rating 1.6219698599799943
play_star_rating val_star_rating 1.3064820362989538
star_rating val_star_rating 2.1512244035203874


Note that there is substantial correlation between the `play_star_rating`, `star_rating` and `val_star_rating`. While this could lead to multicollinearity in your eventual regression model, it is too early to clearly determine this at this point. Remember that multicollinearity is a relationship between 3 or more variables while correlation simply investigates the relationship between two variables.

Additionally, these relationships provide an alternative method for imputing missing values: since they appear to be correlated, you could use these features to help impute missing values in the others features. For example, if you are missing the star_rating for a particular row but have the val_star_rating for that same entry, it seems reasonable to assume that it is a good estimate for the missing star_rating value as they are highly correlated. That said, doing so does come with risks; indeed you would be further increasing the correlation between these features which could further provoke multicollinearity in the final model.

Investigate if you could use one of the other star rating features when one is missing. How many rows have one of `play_star_rating`, `star_rating` and `val_star_rating` missing, but not all three.

In [26]:
len(df[(df["play_star_rating"].isna() |\
df["star_rating"].isna() |\
df["val_star_rating"].isna()              
  
) 
  ])



1569

In [16]:
len(df[(
                                   
(df["play_star_rating"].isna() &\
df["star_rating"].isna() &\
df["val_star_rating"].isna() 
)                                
                                   
  
) ])

1421

Well, it seems like when one is missing, the other two are also apt to be missing. While this has been a bit of an extended investigation, simply go ahead and fill the missing values with that features median.  

Fill in the missing `review_difficulty` values with 'unknown'.

In [17]:
df["review_difficulty"] = df["review_difficulty"].fillna(value="Unknown")

In [18]:
df["play_star_rating"] = df["play_star_rating"].fillna(value=df["play_star_rating"])
df["star_rating"] = df["star_rating"].fillna(value=df["star_rating"])
df["val_star_rating"] = df["val_star_rating"].fillna(value=df["val_star_rating"])


## Normalizing the Data

Now, you'll need to convert all of our numeric columns to the same scale by **_normalizing_** our dataset.  Recall that you normalize a dataset by converting each numeric value to it's corresponding z-score for the column, which is obtained by subtracting the column's mean and then dividing by the column's standard deviation for every value. 


In the cell below:

* Normalize the numeric X features by subtracting the column mean and dividing by the column standard deviation. 
(Don't bother to normalize the list_price as this is the feature you will be predicting.)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10870 entries, 0 to 10869
Data columns (total 14 columns):
prod_id              10870 non-null int64
ages                 10870 non-null object
piece_count          10870 non-null int64
set_name             10870 non-null object
prod_desc            10512 non-null object
prod_long_desc       10870 non-null object
theme_name           10870 non-null object
country              10870 non-null object
list_price           10870 non-null float64
num_reviews          9449 non-null float64
play_star_rating     9321 non-null float64
review_difficulty    10870 non-null object
star_rating          9449 non-null float64
val_star_rating      9301 non-null float64
dtypes: float64(5), int64(2), object(7)
memory usage: 1.2+ MB


In [21]:
df["list_price_n"] = (df["list_price"] - df["list_price"].mean())/df["list_price"].std()

In [22]:
df["num_reviews_n"] = (df["num_reviews"] - df["num_reviews"].mean())/df["num_reviews"].std()

In [24]:
df["play_star_rating_n"] = (df["play_star_rating"] - df["play_star_rating"].mean())/df["play_star_rating"].std()

In [28]:
df["star_rating_n"] = (df["star_rating"] - df["star_rating"].mean())/df["star_rating"].std()

In [29]:
df["val_star_rating_n"] = (df["val_star_rating"] - df["val_star_rating"].mean())/df["val_star_rating"].std()

## Saving Your Results

While you'll once again practice one-hot encoding as you would to preprocess data before fitting a model, saving such a reperesentation of the data will eat up additional disk space. After all, a categorical variable with 10 bins will be transformed to 10 seperate features when passed through `pd.get_dummies()`. As such, while the further practice is worthwhile, save your DataFrame as is for now.

In [31]:
df.to_json("llew.json")

In [32]:
df.head()

,prod_id,ages,piece_count,set_name,prod_desc,prod_long_desc,theme_name,country,list_price,num_reviews,play_star_rating,review_difficulty,star_rating,val_star_rating,list_price_n,num_reviews_n,play_star_rating_n,star_rating_n,val_star_rating_n
0,75823,6-12,277,Bird Island Egg Heist,Catapult into action and take back the eggs fr...,Use the staircase catapult to launch Red into ...,Angry Birds™,US,29.99,2.0,4.0,Average,4.5,4.0,-0.394216,-0.414333,-0.575779,-0.019979,-0.319626
1,75822,6-12,168,Piggy Plane Attack,Launch a flying attack and rescue the eggs fro...,Pilot Pig has taken off from Bird Island with ...,Angry Birds™,US,19.99,2.0,4.0,Easy,5.0,4.0,-0.499846,-0.414333,-0.575779,0.948144,-0.319626
2,75821,6-12,74,Piggy Car Escape,Chase the piggy with lightning-fast Chuck and ...,Pitch speedy bird Chuck against the Piggy Car....,Angry Birds™,US,12.99,11.0,4.3,Easy,4.3,4.1,-0.573788,-0.178526,-0.089770,-0.407229,-0.170574
3,21030,12+,1032,United States Capitol Building,Explore the architecture of the United States ...,Discover the architectural secrets of the icon...,Architecture,US,99.99,23.0,3.6,Average,4.6,4.3,0.345200,0.135885,-1.223792,0.173645,0.127530
4,21035,12+,744,Solomon R. Guggenheim Museum®,Recreate the Solomon R. Guggenheim Museum® wit...,Discover the architectural secrets of Frank Ll...,Architecture,US,79.99,14.0,3.2,Challenging,4.6,4.1,0.133938,-0.099923,-1.871805,0.173645,-0.170574


## One-Hot Encoding Categorical Columns

As a final step, you'll need to deal with the categorical columns by **_one-hot encoding_** them into binary variables via the `pd.get_dummies()` method.  

When doing this, you should also subset to appropriate features. If you were to simply pass the entire DataFrame to the `pd.get_dummies()` method as it stands now, then you would end up with unique features for every single product description! (Presumably the descriptions are unique.) As such, you should first subset to the numeric features that you will eventually use in a model along with categorical variables that are not unique.

In the cell below, subset to the appropriate predictive features and then use the [`pd.get_dummies()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) to one-hot encode the dataset.

In [33]:

pd.get_dummies(df,columns=["theme_name", "set_name", "ages"])

,prod_id,piece_count,prod_desc,prod_long_desc,country,list_price,num_reviews,play_star_rating,review_difficulty,star_rating,...,ages_7+,ages_7-12,ages_7-14,ages_8+,ages_8-12,ages_8-14,ages_9+,ages_9-12,ages_9-14,ages_9-16
0,75823,277,Catapult into action and take back the eggs fr...,Use the staircase catapult to launch Red into ...,US,29.99,2.0,4.0,Average,4.5,...,0,0,0,0,0,0,0,0,0,0
1,75822,168,Launch a flying attack and rescue the eggs fro...,Pilot Pig has taken off from Bird Island with ...,US,19.99,2.0,4.0,Easy,5.0,...,0,0,0,0,0,0,0,0,0,0
2,75821,74,Chase the piggy with lightning-fast Chuck and ...,Pitch speedy bird Chuck against the Piggy Car....,US,12.99,11.0,4.3,Easy,4.3,...,0,0,0,0,0,0,0,0,0,0
3,21030,1032,Explore the architecture of the United States ...,Discover the architectural secrets of the icon...,US,99.99,23.0,3.6,Average,4.6,...,0,0,0,0,0,0,0,0,0,0
4,21035,744,Recreate the Solomon R. Guggenheim Museum® wit...,Discover the architectural secrets of Frank Ll...,US,79.99,14.0,3.2,Challenging,4.6,...,0,0,0,0,0,0,0,0,0,0
5,21039,597,Celebrate Shanghai with this LEGO® Architectur...,Recreate Shanghai in China's blend of historic...,US,59.99,7.0,3.7,Average,4.9,...,0,0,0,0,0,0,0,0,0,0
6,21028,598,Celebrate New York City with this LEGO® Archit...,Celebrate the architectural diversity of New Y...,US,59.99,37.0,3.7,Average,4.2,...,0,0,0,0,0,0,0,0,0,0
7,21029,780,Recreate Buckingham Palace with LEGO® Architec...,Build a LEGO® brick model of London's official...,US,49.99,24.0,4.4,Average,4.7,...,0,0,0,0,0,0,0,0,0,0
8,21034,468,Celebrate London with this LEGO® Architecture ...,Celebrate the architectural diversity of Londo...,US,39.99,23.0,3.6,Average,4.7,...,0,0,0,0,0,0,0,0,0,0
9,21033,444,Celebrate Chicago with this LEGO® Architecture...,Recreate Chicago’s blend of historical and mod...,US,39.99,11.0,3.6,Average,4.8,...,0,0,0,0,0,0,0,0,0,0


That's it! You've now successfully scrubbed your dataset--you're now ready for data exploration and modeling!

## Summary

In this lesson, you learned gain practice with data cleaning by:

* Casting columns to the appropriate data types
* Identifying and deal with null values appropriately
* Removing unnecessary columns
* Checking for and deal with multicollinearity
* Normalizing your data